In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [2]:

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
webpage = Website("https://en.chessbase.com")

In [9]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


In [11]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [13]:

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
messages = messages_for(webpage)

In [16]:
messages

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Chess News | ChessBase\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nShop\nHome\nOPENINGS A-Z\nSupport\nCB Magazine\nSupport-Center\nDownload\nNewsletter\nLearn chess\nHome\nOPENINGS A-Z\nCB Magazine\nCB Books\nSupport\nSupport-Center\nNewsletter\nLearn chess\nToggle navigation\nHome\nOPENINGS A-Z\nCB Magazine\nCB Books\nSupport-Center\nDownload\nNewsletter\nLearn chess\nLogin\nPLAYCHESS\nTACTICS\nVIDEO\nFRITZ\nLIVE\nDATABASE\nMYGAMES\nPLAYERS\nSTUDIES\nSHOP\nChess News\nSHOP\nSearch:\nLanguage:\nSearch:\nLanguage:\nDE\nEN\nES\nFR\nLanguage:\nSHOP\nenglish\ndeutsch\nespanol\nfrançaise\nTesting CheckMat

In [19]:


def summarize(url):
    website = Website(url)
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

    response = ollama_via_openai.chat.completions.create(
    model="llama3.2",
    messages=messages)
    
    return response.choices[0].message.content

In [20]:
summarize("https://en.chessbase.com")

"**ChessBase News Headlines**\n\n### Chess News 01.03.2024\n\n#### - \nChess news for the discerning chess enthusiast\nChessBase is a leading provider of online services specifically created for chess players: \n- ChessBase offers access to its vast database with more than two billion positions from over four million games, as well as coverage of major tournaments.\n\nThe best chess courses:\nFor beginners, our interactive introduction to chess will teach you everything.\nGet access to in-depth analysis of the top games using [our ChessBase account] and get an extra one year for free. \nFor experts: Our latest videos provide high-quality information:\n\nAnalyzing tactics in chess with videos\n\nChess Tactics Training with Videos\n\n#### - \nOur news section is the perfect place for you if:\n*   You're new to this section, it's our regularly updated overview on ChessBase events\n*   You want to get informed about upcoming chess tournaments\n*   You have questions and need to know how yo

In [21]:

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


In [22]:
display_summary("https://en.chessbase.com")

This appears to be a ChessBase news page, providing updates and information on various chess-related topics. Here's a summary of the articles featured:

1. **ChessBase 18 Premium-Package**: An introduction to the new premium package, which includes tutorials and analysis tools for chess players of all levels.
2. **The Funky Löwenthal Variation in 60 Minutes**: A 60-minute video course focused on the Löwenthal Variation, a popular chess opening that can be used for both aggressive and positional play.
3. **World Chess Championship 2024: Post-Event Analysis**: An overview of the recent World Rapid and Blitz Championships, including analysis of games played by top players like Carlsen, Nepo, and Pragg.
4. **Fabiano Caruana Qualifies for 2026 Candidates Tournament**: News that Fabiano Caruana has earned a spot in the 2026 Candidates Tournament through his performance on the FIDE Circuit 2024.
5. **Xue Haowen Wins Hastings Congress**: A report on the recent Hastings International Chess Congress, where Chinese player Xue Haowen emerged as the winner of the 98th edition of the tournament.

Additionally, there are various other news articles and advertisements for chess-related products and services.